### Importing libraries

In [1]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = pd.read_excel("online_retail_II.xlsx",
                    sheet_name="Year 2010-2011")

### Data Understanding

In [4]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.info)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Describe #####################")
    print(dataframe.describe().T)

In [5]:
check_df(df, 5)

##################### Shape #####################
(541910, 8)
##################### Types #####################
<bound method DataFrame.info of        Invoice StockCode                          Description  Quantity         InvoiceDate   Price  Customer ID         Country
0       536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00  2.5500   17850.0000  United Kingdom
1       536365     71053                  WHITE METAL LANTERN         6 2010-12-01 08:26:00  3.3900   17850.0000  United Kingdom
2       536365    84406B       CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00  2.7500   17850.0000  United Kingdom
3       536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6 2010-12-01 08:26:00  3.3900   17850.0000  United Kingdom
4       536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6 2010-12-01 08:26:00  3.3900   17850.0000  United Kingdom
...        ...       ...                                  ...       ...           

### Data Preprocessing

In [6]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [7]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [8]:
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

In [9]:
df.dropna(inplace=True)

In [10]:
df = df[~df["Invoice"].str.contains("C", na=False)]

In [11]:
df = df[df["Quantity"] > 0]

### After preprocessing

In [12]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.0000,11.7007,24.1262,1.0000,2.0000,6.0000,12.0000,253.0000
Price,397925.0000,2.9028,3.3368,0.0000,1.2500,1.9500,3.7500,44.7150
Customer ID,397925.0000,15294.3086,1713.1727,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000


In [13]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [14]:
today_date = dt.datetime(2011, 12, 11)

In [15]:
df = df[df["Country"] == "United Kingdom"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0000,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom,15.3000
1,536365,71053,WHITE METAL LANTERN,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0000,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom,22.0000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400


### Preparation of Lifetime Data Structure

In [16]:
cltv_df = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                         lambda date: (today_date - date.min()).days],
                                         'Invoice': lambda num: num.nunique(),
                                         'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

In [17]:
cltv_df

InvoiceDate             Invoice TotalPrice
             <lambda_0> <lambda_1> <lambda>   <lambda>
Customer ID                                           
12346.0000            0        326        1   263.1200
12747.0000          366        370       11  4196.0100
12748.0000          372        374      210 32348.7750
12749.0000          209        214        5  4085.5950
12820.0000          323        327        4   942.3400
...                 ...        ...      ...        ...
18280.0000            0        278        1   180.6000
18281.0000            0        181        1    80.8200
18282.0000          118        127        2   178.0500
18283.0000          333        338       16  2094.8800
18287.0000          158        202        3  1837.2800

[3921 rows x 4 columns]

In [18]:
cltv_df.columns = cltv_df.columns.droplevel(0)

In [19]:
cltv_df

,<lambda_0>,<lambda_1>,<lambda>,<lambda>
Customer ID,,,,
12346.0000,0,326,1,263.1200
12747.0000,366,370,11,4196.0100
12748.0000,372,374,210,32348.7750
12749.0000,209,214,5,4085.5950
12820.0000,323,327,4,942.3400
...,...,...,...,...
18280.0000,0,278,1,180.6000
18281.0000,0,181,1,80.8200
18282.0000,118,127,2,178.0500


In [20]:
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']
cltv_df

,recency,T,frequency,monetary
Customer ID,,,,
12346.0000,0,326,1,263.1200
12747.0000,366,370,11,4196.0100
12748.0000,372,374,210,32348.7750
12749.0000,209,214,5,4085.5950
12820.0000,323,327,4,942.3400
...,...,...,...,...
18280.0000,0,278,1,180.6000
18281.0000,0,181,1,80.8200
18282.0000,118,127,2,178.0500


In [21]:
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]

In [22]:
cltv_df = cltv_df[cltv_df["monetary"] > 0]
cltv_df.head()

,recency,T,frequency,monetary
Customer ID,,,,
12346.0000,0,326,1,263.1200
12747.0000,366,370,11,381.4555
12748.0000,372,374,210,154.0418
12749.0000,209,214,5,817.1190
12820.0000,323,327,4,235.5850


In [23]:
cltv_df["recency"] = cltv_df["recency"] / 7
cltv_df["T"] = cltv_df["T"] / 7

In [24]:
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

### Establishment of BG-NBD Model

In [25]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

In [26]:
bgf.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'])

<lifetimes.BetaGeoFitter: fitted with 2570 subjects, a: 0.12, alpha: 11.66, b: 2.51, r: 2.21>

### Who are the 10 customers we expect the most to purchase in a week?

In [27]:
bgf.predict(1,
            cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T']).sort_values(ascending=False).head(10)

Customer ID
12748.0000   3.2530
17841.0000   1.9333
13089.0000   1.5320
14606.0000   1.4588
15311.0000   1.4286
12971.0000   1.3523
13408.0000   0.9829
18102.0000   0.9653
13798.0000   0.9082
14527.0000   0.8830
dtype: float64

In [28]:
cltv_df["expected_purc_1_week"] = bgf.predict(1,
                                              cltv_df['frequency'],
                                              cltv_df['recency'],
                                              cltv_df['T'])

In [29]:
cltv_df.head()

,recency,T,frequency,monetary,expected_purc_1_week
Customer ID,,,,,
12747.0000,52.2857,52.8571,11,381.4555,0.2025
12748.0000,53.1429,53.4286,210,154.0418,3.2530
12749.0000,29.8571,30.5714,5,817.1190,0.1671
12820.0000,46.1429,46.7143,4,235.5850,0.1040
12822.0000,2.2857,12.5714,2,474.4400,0.1291


### Who are the 10 customers we expect to purchase the most in 1 month?

In [30]:
bgf.predict(4,
            cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T']).sort_values(ascending=False).head(10)

Customer ID
12748.0000   12.9775
17841.0000    7.7126
13089.0000    6.1117
14606.0000    5.8198
15311.0000    5.6994
12971.0000    5.3947
13408.0000    3.9212
18102.0000    3.8508
13798.0000    3.6233
14527.0000    3.5224
dtype: float64

In [31]:
cltv_df["expected_purc_1_month"] = bgf.predict(4,
                                               cltv_df['frequency'],
                                               cltv_df['recency'],
                                               cltv_df['T'])

In [32]:
cltv_df.sort_values("expected_purc_1_month", ascending=False).head(20)

,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month
Customer ID,,,,,,
12748.0000,53.1429,53.4286,210,154.0418,3.2530,12.9775
17841.0000,53.0000,53.4286,124,330.5490,1.9333,7.7126
13089.0000,52.2857,52.8571,97,605.9323,1.5320,6.1117
14606.0000,53.1429,53.4286,93,130.2835,1.4588,5.8198
15311.0000,53.2857,53.4286,91,667.6176,1.4286,5.6994
12971.0000,52.5714,53.2857,86,127.0891,1.3523,5.3947
13408.0000,53.0000,53.4286,62,453.5006,0.9829,3.9212
18102.0000,52.2857,52.5714,60,3613.4663,0.9653,3.8508
13798.0000,52.8571,53.2857,57,650.5972,0.9082,3.6233


### Establishing the GAMMA-GAMMA Model

In [33]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                        cltv_df['monetary']).head(10)

Customer ID
12747.0000   387.8251
12748.0000   154.1852
12749.0000   846.8301
12820.0000   247.0851
12822.0000   520.8465
12823.0000   365.1374
12826.0000   216.4986
12827.0000   153.3754
12828.0000   175.4090
12829.0000   162.2466
dtype: float64

In [34]:
ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                        cltv_df['monetary']).sort_values(ascending=False).head(10)

Customer ID
14088.0000   3919.3411
18102.0000   3623.8567
14096.0000   3201.8693
17511.0000   2939.4746
15749.0000   2723.5999
17450.0000   2661.5770
13081.0000   2616.6981
16984.0000   2452.2114
16000.0000   2208.0329
16684.0000   2143.0599
dtype: float64

In [35]:
cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                             cltv_df['monetary'])

In [36]:
cltv_df.sort_values("expected_average_profit", ascending=False).head(20)

,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit
Customer ID,,,,,,,
14088.0000,44.5714,46.1429,13,3867.4988,0.2597,1.0357,3919.3411
18102.0000,52.2857,52.5714,60,3613.4663,0.9653,3.8508,3623.8567
14096.0000,13.8571,14.5714,17,3169.4421,0.7231,2.8735,3201.8693
17511.0000,52.8571,53.4286,31,2923.1394,0.5074,2.0241,2939.4746
15749.0000,13.8571,47.5714,3,2567.1067,0.0284,0.1132,2723.5999
17450.0000,51.2857,52.5714,46,2651.6015,0.7451,2.9723,2661.5770
13081.0000,51.2857,53.1429,11,2575.6800,0.2009,0.8016,2616.6981
16984.0000,5.8571,18.7143,2,2240.6750,0.1033,0.4107,2452.2114
16000.0000,0.0000,0.4286,3,2080.9267,0.4160,1.6409,2208.0329


### 1 Month CLTV Prediction

In [37]:
cltv_1_month = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=1,
                                   freq="W",
                                   discount_rate=0.01)


In [38]:
cltv_1_month = cltv_1_month.reset_index()
cltv_final_1_month = cltv_df.merge(cltv_1_month, on="Customer ID", how="left")

In [39]:
# Standardization of CLTV

scaler = MinMaxScaler(feature_range=(0, 5))
scaler.fit(cltv_final_1_month[["clv"]])
cltv_final_1_month["scaled_clv"] = scaler.transform(cltv_final_1_month[["clv"]])
cltv_final_1_month.sort_values(by="scaled_clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,scaled_clv
2486,18102.0000,52.2857,52.5714,60,3613.4663,0.9653,3.8508,3623.8567,15003.6515,5.0000
589,14096.0000,13.8571,14.5714,17,3169.4421,0.7231,2.8735,3201.8693,9888.2529,3.2953
2184,17450.0000,51.2857,52.5714,46,2651.6015,0.7451,2.9723,2661.5770,8505.5738,2.8345
2213,17511.0000,52.8571,53.4286,31,2923.1394,0.5074,2.0241,2939.4746,6396.9361,2.1318
1804,16684.0000,50.4286,51.2857,28,2129.8382,0.4767,1.9015,2143.0599,4381.2040,1.4600
587,14088.0000,44.5714,46.1429,13,3867.4988,0.2597,1.0357,3919.3411,4364.4185,1.4545
406,13694.0000,52.7143,53.4286,50,1268.2560,0.7983,3.1845,1272.6835,4357.5436,1.4522
1173,15311.0000,53.2857,53.4286,91,667.6176,1.4286,5.6994,668.9157,4098.9999,1.3660
133,13089.0000,52.2857,52.8571,97,605.9323,1.5320,6.1117,607.0411,3988.9627,1.3293
1485,16000.0000,0.0000,0.4286,3,2080.9267,0.4160,1.6409,2208.0329,3891.0472,1.2967


### 12 Month CLTV Prediction

In [40]:
cltv_12_month = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=12,
                                   freq="W",
                                   discount_rate=0.01)

In [41]:
cltv_12_month = cltv_12_month.reset_index()
cltv_final_12_month = cltv_df.merge(cltv_12_month, on="Customer ID", how="left")

In [42]:
# Standardization of CLTV

scaler = MinMaxScaler(feature_range=(0, 5))
scaler.fit(cltv_final_12_month[["clv"]])
cltv_final_12_month["scaled_clv"] = scaler.transform(cltv_final_12_month[["clv"]])
cltv_final_12_month.sort_values(by="scaled_clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,scaled_clv
2486,18102.0000,52.2857,52.5714,60,3613.4663,0.9653,3.8508,3623.8567,164895.4238,5.0000
589,14096.0000,13.8571,14.5714,17,3169.4421,0.7231,2.8735,3201.8693,105245.0028,3.1913
2184,17450.0000,51.2857,52.5714,46,2651.6015,0.7451,2.9723,2661.5770,93472.4353,2.8343
2213,17511.0000,52.8571,53.4286,31,2923.1394,0.5074,2.0241,2939.4746,70314.3608,2.1321
1804,16684.0000,50.4286,51.2857,28,2129.8382,0.4767,1.9015,2143.0599,48111.6391,1.4589
406,13694.0000,52.7143,53.4286,50,1268.2560,0.7983,3.1845,1272.6835,47905.7062,1.4526
587,14088.0000,44.5714,46.1429,13,3867.4988,0.2597,1.0357,3919.3411,47786.6750,1.4490
1173,15311.0000,53.2857,53.4286,91,667.6176,1.4286,5.6994,668.9157,45069.2433,1.3666
133,13089.0000,52.2857,52.8571,97,605.9323,1.5320,6.1117,607.0411,43849.4604,1.3296
1057,15061.0000,52.5714,53.2857,48,1110.0481,0.7693,3.0690,1114.0952,40412.2241,1.2254


### 6 Aylık CLTV Prediction

In [44]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=6,  # 6 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

cltv.head()

Customer ID
12747.0000    1937.0576
12748.0000   12382.9721
12749.0000    3457.1778
12820.0000     631.9499
12822.0000    1612.1533
Name: clv, dtype: float64

In [45]:
cltv = cltv.reset_index()
cltv.sort_values(by="clv", ascending=False).head(50)
cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")

cltv_final.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv
2486,18102.0000,52.2857,52.5714,60,3613.4663,0.9653,3.8508,3623.8567,86333.8981
589,14096.0000,13.8571,14.5714,17,3169.4421,0.7231,2.8735,3201.8693,55833.4390
2184,17450.0000,51.2857,52.5714,46,2651.6015,0.7451,2.9723,2661.5770,48940.7431
2213,17511.0000,52.8571,53.4286,31,2923.1394,0.5074,2.0241,2939.4746,36812.0366
1804,16684.0000,50.4286,51.2857,28,2129.8382,0.4767,1.9015,2143.0599,25198.9202
406,13694.0000,52.7143,53.4286,50,1268.2560,0.7983,3.1845,1272.6835,25078.4061
587,14088.0000,44.5714,46.1429,13,3867.4988,0.2597,1.0357,3919.3411,25061.3555
1173,15311.0000,53.2857,53.4286,91,667.6176,1.4286,5.6994,668.9157,23592.1401
133,13089.0000,52.2857,52.8571,97,605.9323,1.5320,6.1117,607.0411,22955.9564
1485,16000.0000,0.0000,0.4286,3,2080.9267,0.4160,1.6409,2208.0329,21369.2996


In [46]:
# Standardization of CLTV
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(cltv_final[["clv"]])
cltv_final["scaled_clv"] = scaler.transform(cltv_final[["clv"]])

In [47]:
# Sorting 
cltv_final.sort_values(by="scaled_clv", ascending=False).head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,scaled_clv
2486,18102.0000,52.2857,52.5714,60,3613.4663,0.9653,3.8508,3623.8567,86333.8981,1.0000
589,14096.0000,13.8571,14.5714,17,3169.4421,0.7231,2.8735,3201.8693,55833.4390,0.6467
2184,17450.0000,51.2857,52.5714,46,2651.6015,0.7451,2.9723,2661.5770,48940.7431,0.5669
2213,17511.0000,52.8571,53.4286,31,2923.1394,0.5074,2.0241,2939.4746,36812.0366,0.4264
1804,16684.0000,50.4286,51.2857,28,2129.8382,0.4767,1.9015,2143.0599,25198.9202,0.2919


### Creating Segments by CLTV

In [48]:
cltv_final["segment"] = pd.qcut(cltv_final["scaled_clv"], 4, labels=["D", "C", "B", "A"])

In [49]:
cltv_final.head()

,Customer ID,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit,clv,scaled_clv,segment
0,12747.0000,52.2857,52.8571,11,381.4555,0.2025,0.8077,387.8251,1937.0576,0.0224,A
1,12748.0000,53.1429,53.4286,210,154.0418,3.2530,12.9775,154.1852,12382.9721,0.1434,A
2,12749.0000,29.8571,30.5714,5,817.1190,0.1671,0.6657,846.8301,3457.1778,0.0400,A
3,12820.0000,46.1429,46.7143,4,235.5850,0.1040,0.4146,247.0851,631.9499,0.0073,C
4,12822.0000,2.2857,12.5714,2,474.4400,0.1291,0.5127,520.8465,1612.1533,0.0187,B


In [50]:
cltv_final.groupby("segment").agg(
    {"count", "mean", "sum"})

Customer ID                     recency                        T                  frequency             monetary                   expected_purc_1_week                expected_purc_1_month                expected_average_profit                         clv                    scaled_clv              
               mean           sum count    mean        sum count    mean        sum count      mean   sum count     mean         sum count                 mean      sum count                  mean      sum count                    mean         sum count      mean          sum count       mean     sum count
segment                                                                                                                                                                                                                                                                                                            
D        15706.1788 10099073.0000   643 22.0671 14189.1429   643 40.5090 26047.2857   643    3.0684  1973   643 178.2870 114638.5415   643               0.0710  45.6644   643                0.2829 181.8727   643                193.1456 124192.6240   643  270.2260  173755.2867   643     0.0031  2.0126   643
C        15521.4050  9964742.0000   642 30.8364 19797.0000   642 38.1656 24502.2857   642    3.9907  2562   642 261.1129 167634.5001   642               0.1195  76.7253   642                0.4759 305.5425   642                278.5283 178815.1409   642  711.9518  457073.0410   642     0.0082  5.2942   642
B        15596.3364 10012848.0000   642 29.8389 19156.5714   642 35.1170 22545.1429   642    5.4548  3502   642 352.4795 226291.8641   642               0.1614 103.6310   642                0.6425 412.4838   642                371.0177 238193.3792   642 1273.2854  817449.2097   642     0.0147  9.4685   642
A        15390.8383  9896309.0000   643 31.4606 20229.1429   643 34.5192 22195.8571   643   11.2924  7261   643 587.2369 377593.3034   643               0.2724 175.1616   643                1.0846 697.3743   643                609.6752 392021.1662   643 3813.3658 2451994.2197   643     0.0442 28.4013   643